In [1]:
# Setup environment
from pyspark.sql import Row

StatementMeta(, 26a0a09c-d1f2-4fda-b0fa-d6993487e207, 3, Finished, Available, Finished)

In [2]:
# Read in data from delta tables
table_of_interest = "NHL_Lakehouse_Bronze.nhl_game_plays" # Define table of interest
df = spark.read.table(table_of_interest)

StatementMeta(, 26a0a09c-d1f2-4fda-b0fa-d6993487e207, 4, Finished, Available, Finished)

In [3]:
print(df.count())

StatementMeta(, 26a0a09c-d1f2-4fda-b0fa-d6993487e207, 5, Finished, Available, Finished)

5050529


In [4]:
# Declare missing eventid
missing_play_ids =    ["2011020767_23", "2012030135_359", "2013020459_258",
                        "2010020237_29", "2011021075_138", "2010020863_291",
                        "2011020642_5", "2015020367_33", "2018020963_83",
                        "2011030126_388", "2018020573_216"]

StatementMeta(, 26a0a09c-d1f2-4fda-b0fa-d6993487e207, 6, Finished, Available, Finished)

In [5]:
df.columns
df.printSchema()

StatementMeta(, 26a0a09c-d1f2-4fda-b0fa-d6993487e207, 7, Finished, Available, Finished)

root
 |-- play_id: string (nullable = true)
 |-- game_id: integer (nullable = true)
 |-- team_id_for: string (nullable = true)
 |-- team_id_against: string (nullable = true)
 |-- event: string (nullable = true)
 |-- secondarytype: string (nullable = true)
 |-- x: string (nullable = true)
 |-- y: string (nullable = true)
 |-- period: integer (nullable = true)
 |-- periodtype: string (nullable = true)
 |-- periodtime: integer (nullable = true)
 |-- periodtimeremaining: string (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- goals_away: integer (nullable = true)
 |-- goals_home: integer (nullable = true)
 |-- description: string (nullable = true)
 |-- st_x: string (nullable = true)
 |-- st_y: string (nullable = true)



In [6]:
# Your list of missing ids (already defined)
# missing_play_ids = [...]

# 1) Build a DF with one row per missing play_id:
#    - play_id filled
#    - event = "Missing Details"
#    - everything else = NULL
cols = df.columns

rows = [
    Row(
        **{
            c: (
                pid if c == "play_id"
                else "Missing Details" if c == "event"
                else None
            )
            for c in cols
        }
    )
    for pid in missing_play_ids
]

missing_df = spark.createDataFrame(rows, schema=df.schema)

# 2) (Recommended) Only keep truly missing IDs to avoid duplicates
existing_ids = df.select("play_id").distinct()
missing_df = missing_df.join(existing_ids, on="play_id", how="left_anti")

# 3) Append the records
df_final = df.unionByName(missing_df)

StatementMeta(, 26a0a09c-d1f2-4fda-b0fa-d6993487e207, 8, Finished, Available, Finished)

In [7]:
print(df_final.tail(12))
print(df.count())
print(df_final.count())

StatementMeta(, 26a0a09c-d1f2-4fda-b0fa-d6993487e207, 9, Finished, Available, Finished)

[Row(play_id='2018030417_301', game_id=2018030417, team_id_for='NA', team_id_against='NA', event='Game End', secondarytype='NA', x='NA', y='NA', period=3, periodtype='REGULAR', periodtime=1200, periodtimeremaining='0', datetime=datetime.datetime(2019, 6, 13, 3, 47, 2), goals_away=4, goals_home=1, description='Game End', st_x='NA', st_y='NA'), Row(play_id='2011020767_23', game_id=None, team_id_for=None, team_id_against=None, event='Missing Details', secondarytype=None, x=None, y=None, period=None, periodtype=None, periodtime=None, periodtimeremaining=None, datetime=None, goals_away=None, goals_home=None, description=None, st_x=None, st_y=None), Row(play_id='2012030135_359', game_id=None, team_id_for=None, team_id_against=None, event='Missing Details', secondarytype=None, x=None, y=None, period=None, periodtype=None, periodtime=None, periodtimeremaining=None, datetime=None, goals_away=None, goals_home=None, description=None, st_x=None, st_y=None), Row(play_id='2013020459_258', game_id=No

In [8]:
# Write cleaned data into lakehouse

( df_final
    .write.format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable("NHL_Lakehouse_Silver.silver_events")
)

StatementMeta(, 26a0a09c-d1f2-4fda-b0fa-d6993487e207, 10, Finished, Available, Finished)